# Compile the main simulator code using cython 

In [ ]:
!python ../setup.py build_ext --inplace

# Import all the relevant files 

In [ ]:
# Import the pipeline creation object
import sys
sys.path.append('../')

# Import the pipeline creation object
import pandas as pd
import numpy as np
import common.cbgt as cbgt
import common.pipeline_creation as pl_creat
import common.plotting_functions as plt_func
import common.plotting_helper_functions as plt_help
import common.postprocessing_helpers as post_help
import importlib

importlib.reload(plt_help)
importlib.reload(plt_func)
importlib.reload(post_help)


%load_ext autoreload
%autoreload 2
%reload_ext autoreload 

import warnings
warnings.simplefilter('ignore', category=FutureWarning)

# Choose the experiment and create the main pipeline

In [ ]:
# Choose the experiment:
experiment_choice = 'n-choice'

if experiment_choice == 'stop-signal':
    import stopsignal.paramfile_stopsignal as paramfile
elif experiment_choice == 'n-choice':
    import nchoice.paramfile_nchoice as paramfile

number_of_choices = 3

#Choose which multiprocessing library to use
use_library = "pathos" # "none" or "pathos" or "ray"

# Call choose_pipeline with the pipeline object
pl_creat.choose_pipeline(experiment_choice)

#Create the main pipeline
pl = pl_creat.create_main_pipeline(runloop=True)

# how many simulations do you want to run ? each simulation is executed as a seperate thread
num_sims = 1
num_cores = 7
# set a seed
import time
np.random.seed(int(time.time()))
#seed = np.random.choice(np.random.randint(0,9999999,100),1)[0]
seed = 8867366
print(seed)


In [ ]:

# Define the data and figure directory

data_dir = "../Data/"
figure_dir = "../Figures/"



# Modify cellular parameters as desired. 
### The paramfile has all the parameter dictionaries that can be modified. They are listed as below:
- celldefaults (neuronal parameters)
- d1defaults (dSPN parameters)
- d2defaults (iSPN parameters)
- dpmndefaults (dopamine related parameters)
- basestim (background input for the nuclei)
- popspecific (population specific parameters)
- receptordefaults (GABA, AMPA receptor parameters)
### The details of each of these dictionaries can be checked by simply typing paramfile.<parameter name> as also shown in the block below

In [ ]:
# list out the available parameter dictionaries
dir(paramfile)

In [ ]:
# view (or edit) one of the parameter dictionaries
paramfile.celldefaults

# Running the pipeline

### 1. Default values of environment variable

In [ ]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
configuration = {
    'seed': seed,
    'experimentchoice': experiment_choice,
    'inter_trial_interval': None, # default = 600ms
    'thalamic_threshold': None, # default 30sp/s
    'movement_time':None,# #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout':None, #wait for a choice to be made, default 1000, 
    'params': paramfile.celldefaults,  #neuron parameters
    'pops': paramfile.popspecific,    #population parameters
    'receps' : paramfile.receptordefaults, #receptor parameters
    'base' : paramfile.basestim,   #baseline stimulation parameters
    'dpmns' : paramfile.dpmndefaults,  #dopamine related parameters
    'dSPN_params' : paramfile.dSPNdefaults,     #dSPNs population related parameters
    'iSPN_params' : paramfile.iSPNdefaults,     #iSPNs population related parameters
    'channels' : pd.DataFrame([["A"],["B"],["C"]], columns=['action']), #action channels related parameters (init_params.py)
    'number_of_choices':number_of_choices,
    #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' :None,
    'Q_support_params': None,#pd.DataFrame([[85,0.6]],columns=['dpmn_CPP_scale','q_alpha']), #initialization of Q-values update (qvalues.py) 
    'Q_df_set':pd.DataFrame([[0.5, 0.5,0.5]],columns=["A","B","C"]), #initialized Q-values df (qvalues.py)  
    'n_trials': 3, #number of trials (generateepochs.py)
    'volatility': [1,"exact"], #frequency of changepoints and type of change points (exact or poisson) (generateepochs.py)
    'conflict': (0.7, 0.3,0.4), #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.01, #std for the magnitude of the reward (generateepochs.py)
    'maxstim': 0.8, # 3 instead? how much stimulus the LIP gets at maximum
    'corticostriatal_plasticity_present':False, # whether plasticity is on or "off" at corticostriatal synapses
    'record_variables':["weight","optogenetic_input"],#"optogenetic_input"],
    'opt_signal_present':[True, True],
    'opt_signal_probability':[[0],[1]], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude':[0.1, -0.5], # ampitude of the stop signal over base line
    'opt_signal_onset':[10.,10.], # in ms
    'opt_signal_duration':["phase 0",400.],
    'opt_signal_channel': ["all","all"], # "all" (all channels are given the stop signal) 
    'opt_signal_population':["iSPN","dSPN"],
    'sustainedfraction':0.7
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signal
    
}


### Run the simulation - num_sims thread with the environment variable

In [ ]:
# pl.modulelist

In [ ]:
results = cbgt.ExecutionManager(cores=num_cores,use=use_library).run([pl]*num_sims,[configuration]*num_sims)


In [ ]:
# List all the agent variables accessible
results[0].keys()

In [ ]:
results[0]['meaneff_GABA']

In [ ]:
results[0]['conflict']

In [ ]:
results[0]['Q_df']

In [ ]:
experiment_choice

In [ ]:
#Extract all the relevant dataframes
firing_rates, reward_q_df, performance, rt_dist, total_performance = plt_help.extract_relevant_frames(results,seed,experiment_choice)

In [ ]:
results[0]['popfreqs']

In [ ]:
# firing rates  
firing_rates[0]

In [ ]:
performance

In [ ]:
recorded_variables = post_help.extract_recording_variables(results,results[0]['record_variables'],seed)

In [ ]:
recorded_variables['weight']

In [ ]:
recorded_variables['optogenetic_input'].nuclei.unique()

In [ ]:
recorded_variables['optogenetic_input']

In [ ]:
import seaborn as sns
import pylab as pl
sns.pointplot(x="trials",y="weights",hue="nuclei",data=recorded_variables['weight'])


In [ ]:
fig,ax = pl.subplots(1,1,figsize=(5,4))
sns.set(style="white", font_scale=1.2)
sns.lineplot(x="Time(ms)",y="value",data=recorded_variables['optogenetic_input'],hue='nuclei',lw=3.0,ax=ax)
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
ax.legend(loc='upper right',bbox_to_anchor=[1.1, 0.8])
pl.tight_layout()
fig.savefig(figure_dir+'optogenetic_stimulation.png')


In [ ]:
# Relevant reward_dataframe
reward_q_df[0]

In [ ]:
performance

In [ ]:
# Extract the data tables from the agent
datatables = cbgt.collateVariable(results,'datatables')
datatables[0]["decisiondurationplusdelay"]-datatables[0]["decisionduration"]

In [ ]:
datatables[0]

In [ ]:
# Check the Q-values data frame
results[0]['Q_df']

In [ ]:
# postfix = "weak_GPi_inh"
postfix = ""

In [ ]:
#Save the selected variables of results in the data_dir specified
cbgt.saveResults(results,data_dir+'network_data_'+postfix+'_'+str(seed)+'.pickle',['popfreqs','popdata','Q_support_params','dSPN_params','iSPN_params','dpmns','datatables'])


In [ ]:
firing_rates[0].to_csv(data_dir+"firing_rates_"+postfix+".csv")
reward_q_df[0].to_csv(data_dir+"reward_df_"+postfix+".csv")
performance.to_csv(data_dir+"performance_"+postfix+".csv")
rt_dist.to_csv(data_dir+"rt_dist_"+postfix+".csv")
total_performance.to_csv(data_dir+"total_performance_"+postfix+".csv")

In [ ]:
reward_q_df

In [ ]:
results[0]['opt_list_trials_list'][1]

In [ ]:
# Plot the firing rates extracted, figure handles are returned in fig_handles

FR_fig_handles = plt_func.plot_fr(firing_rates,datatables,results,experiment_choice,True)
FR_fig_handles[0].savefig(figure_dir+"Example_FR_optostim_"+postfix+"two_stims.png",dpi=300)

In [ ]:
reward_fig_handles = plt_func.plot_reward_Q_df(reward_q_df)